In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/test (3).csv
/kaggle/input/fake-news/train.csv


In [2]:
import pandas as pd
import numpy as np


In [3]:
data= pd.read_csv('/kaggle/input/fake-news/train.csv')

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
data.dropna(inplace=True)

In [6]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
data.reset_index(inplace=True)

In [8]:
x = data['title']
y = data['label']

In [9]:
print(x.head())
print(y.head())

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object
0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64


In [10]:
import re
import nltk
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,Flatten,Dense,LSTM
from tensorflow.keras.models import Sequential

In [11]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

Preprocessing of text

In [12]:
corpus =[]
for i in range(0,len(x)):
    text = re.sub('[^a-zA-Z]',' ',x[i])
    text = text.lower()
    text = text.split()
    text  = [ps.stem(word) for word in text if word not in stopword]
    text = ' '.join(text)
    corpus.append(text)

In [13]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [14]:
#one_hot_representation---
vocab_length = 5000
one_hot_rep = [one_hot(x,vocab_length) for x in corpus]
one_hot_rep[:10]

[[4753, 2927, 4198, 3294, 4858, 4564, 4996, 717, 758, 1797],
 [4117, 480, 720, 1087, 783, 2507, 394],
 [1719, 2917, 4763, 3301],
 [2409, 143, 2132, 4774, 4062, 3899],
 [4621, 783, 4391, 639, 2068, 1273, 783, 623, 2776, 3730],
 [689,
  850,
  2622,
  139,
  2397,
  605,
  3033,
  852,
  2509,
  3043,
  261,
  1182,
  4394,
  850,
  394],
 [2476, 4264, 33, 1110, 1843, 2766, 1506, 4114, 2328, 4639, 2890],
 [1649, 982, 2543, 857, 184, 673, 605, 3983, 2328, 4639, 2890],
 [1857, 3348, 2998, 4866, 3819, 4554, 3182, 166, 605, 4611],
 [1040, 4959, 2278, 947, 1349, 4632, 3023, 3020]]

In [15]:
#padding----
sent_length = 20
sent_pad = pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)
sent_pad[:10]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4753,
        2927, 4198, 3294, 4858, 4564, 4996,  717,  758, 1797],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 4117,  480,  720, 1087,  783, 2507,  394],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 1719, 2917, 4763, 3301],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 2409,  143, 2132, 4774, 4062, 3899],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4621,
         783, 4391,  639, 2068, 1273,  783,  623, 2776, 3730],
       [   0,    0,    0,    0,    0,  689,  850, 2622,  139, 2397,  605,
        3033,  852, 2509, 3043,  261, 1182, 4394,  850,  394],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 2476, 4264,
          33, 1110, 1843, 2766, 1506, 4114, 2328, 4639, 2890],
       [   0,    0,    0,    0,    0,    

In [16]:
#embedding
model = Sequential()
model.add(Embedding(vocab_length,40,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile('adam','binary_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
len(sent_pad),y.shape

(18285, (18285,))

In [18]:
x_final = np.array(sent_pad)
y_final = np.array(y)

In [19]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [20]:
#fit the model--
model.fit(x_final,y_final,epochs=50,verbose=0)
#evulate the model--
loss,accuracy = model.evaluate(x_final,y_final,verbose=0)
print('Accuracy: %f' % (accuracy*100))
print("loss",loss)

Accuracy: 100.000000
loss 4.0850598459485354e-09


Dropout

In [21]:
#embedding
from tensorflow.keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(vocab_length,40,input_length=sent_length))
#model1.add(Dropout())
model1.add(LSTM(100))
#model1.add(Dropout())
model1.add(Dense(1,activation='sigmoid'))
model1.compile('adam','binary_crossentropy',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.30,random_state=42)

model_training---

In [23]:
model1.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 10s 35ms/step - loss: 0.3235 - accuracy: 0.8501 - val_loss: 0.2006 - val_accuracy: 0.9160
Epoch 2/10
200/200 [==============================] - 7s 33ms/step - loss: 0.1390 - accuracy: 0.9450 - val_loss: 0.1988 - val_accuracy: 0.9176
Epoch 3/10
200/200 [==============================] - 7s 34ms/step - loss: 0.0957 - accuracy: 0.9659 - val_loss: 0.2315 - val_accuracy: 0.9154
Epoch 4/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0669 - accuracy: 0.9762 - val_loss: 0.2579 - val_accuracy: 0.9096
Epoch 5/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0405 - accuracy: 0.9874 - val_loss: 0.3127 - val_accuracy: 0.9069
Epoch 6/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0286 - accuracy: 0.9912 - val_loss: 0.3826 - val_accuracy: 0.9096
Epoch 7/10
200/200 [==============================] - 6s 32ms/step - loss: 0.0175 - accuracy: 0.9943 - val_loss: 0.4332 - val_accuracy: 0.912

In [24]:
from sklearn import metrics
y_pred = (model1.predict(x_test)>=0.5).astype(int)

172/172 [==============================] - 2s 6ms/step


In [25]:
metrics.accuracy_score(y_test,y_pred)

0.9086766314254466

In [26]:
metrics.confusion_matrix(y_test,y_pred)

array([[2832,  275],
       [ 226, 2153]])